In [1]:
%run 2_converter.ipynb
%run 3_dataset.ipynb
%run 4_batching.ipynb
%run 7_model.ipynb

[' ', '!', '"', '#', '%', '&', "'", '(', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'Ó', 'ó', 'Ą', 'ą', 'Ć', 'ć', 'Ę', 'ę', 'Ł', 'ł', 'ń', 'Ś', 'ś', 'ź', 'Ż', 'ż']
['KG', 'G', 'SZT', 'L', 'ML']
['C', 'A', 'B', 'G', '0', 't', 'r', 'c', '4', 'a', '8', 'l', 'k']
[' ', '!', '"', '#', '%', '&', "'", '(', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'Ó', 'ó', 

In [2]:

import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
# -----------------------------------------------------
# LOSS HELPERS
# -----------------------------------------------------

def seq_ce_loss(logits, target, mask, pad_id):
    if mask.sum() == 0:
        return torch.tensor(0.0, device=logits.device)

    logits = logits.reshape(-1, logits.size(-1))
    target = target.reshape(-1)
    mask = mask.reshape(-1)

    return nn.functional.cross_entropy(
        logits[mask],
        target[mask],
        ignore_index=pad_id
    )


def regression_loss(pred, target, present_mask, eps=1e-6):
    present_mask = present_mask.bool()

    if present_mask.sum() == 0:
        return torch.tensor(0.0, device=pred.device)

    # Ensure shapes
    pred = pred.squeeze(-1)
    target = target

    # MSLE = MSE( log(1 + pred) − log(1 + target) )
    pred_safe = torch.clamp(pred[present_mask], min=0)  # MSLE expects non‑negative values
    target_safe = torch.clamp(target[present_mask], min=0)

    log_pred = torch.log1p(pred_safe + eps)
    log_target = torch.log1p(target_safe + eps)

    return nn.functional.mse_loss(log_pred, log_target)

# -----------------------------------------------------
# TRAIN LOOP
# -----------------------------------------------------

def train_one_epoch(model, dataloader, optimizer, device, pad_id, bos_id):
    model.train()
    total_loss = 0.0

    for samples, labels, sample_mask, name_mask in tqdm(dataloader):
        samples = samples.to(device)
        sample_mask = sample_mask.to(device)

        # Build batch label tensors
        names = torch.stack([lab["name"] for lab in labels]).to(device)
        name_mask = name_mask.to(device)

        unit_targets = torch.stack([lab["unit"] for lab in labels]).to(device)
        tax_targets  = torch.stack([lab["tax_category"] for lab in labels]).to(device)

        quantity = torch.stack([lab["quantity"] for lab in labels]).to(device)
        amount = torch.stack([lab["amount"] for lab in labels]).to(device)
        price = torch.stack([lab["price"] for lab in labels]).to(device)
        total_price = torch.stack([lab["total_price"] for lab in labels]).to(device)

        quantity_present = torch.stack([lab["quantity_present"] for lab in labels]).to(device)
        amount_present = torch.stack([lab["amount_present"] for lab in labels]).to(device)
        price_present = torch.stack([lab["price_present"] for lab in labels]).to(device)
        total_price_present = torch.stack([lab["total_price_present"] for lab in labels]).to(device)

        # --------------------------------------------------
        # Build decoder input: BOS + names[:-1]
        # --------------------------------------------------
        bos_column = torch.full((names.size(0), 1), bos_id, dtype=torch.long, device=device)
        decoder_tokens = torch.cat([bos_column, names[:, :-1]], dim=1)

        # --------------------------------------------------
        # Forward
        # --------------------------------------------------

        decoder_mask = torch.cat([
            torch.zeros((names.size(0), 1), dtype=torch.bool, device=device),
            name_mask[:, :-1]
        ], dim=1)

        out = model(
            encoder_tokens=samples,
            decoder_tokens=decoder_tokens,
            encoder_mask=sample_mask,
            decoder_mask=decoder_mask
        )

        name_logits = out["name_logits"]       # (B, T, vocab)
        unit_logits = out["unit_logits"]       # (B, num_units)
        tax_logits  = out["tax_logits"]        # (B, num_tax)
        quantity_pred = out["quantity_pred"]   # (B, 1)
        amount_pred   = out["amount_pred"]     # (B, 1)
        price_pred    = out["price_pred"]      # (B, 1)
        total_pred    = out["total_pred"]      # (B, 1)

        # --------------------------------------------------
        # Losses
        # --------------------------------------------------
        loss_name = seq_ce_loss(name_logits, names, ~name_mask.bool(), pad_id)
        loss_unit = nn.functional.cross_entropy(unit_logits, unit_targets)
        loss_tax  = nn.functional.cross_entropy(tax_logits, tax_targets)

        loss_quantity = regression_loss(quantity_pred, quantity, quantity_present)
        loss_amount   = regression_loss(amount_pred, amount, amount_present)
        loss_price    = regression_loss(price_pred, price, price_present)
        loss_total    = regression_loss(total_pred, total_price, total_price_present)

        loss = (
            loss_name +
            loss_unit +
            loss_tax +
            loss_quantity +
            loss_amount +
            loss_price +
            loss_total
        )

        # print("loss_name: ", loss_name)
        # print("loss_unit: ", loss_unit)
        # print("loss_tax: ", loss_tax)
        # print("loss_quantity: ", loss_quantity)
        # print("loss_amount: ", loss_amount)
        # print("loss_price: ", loss_price)
        # print("loss_total: ", loss_total)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


# -----------------------------------------------------
# MAIN TRAIN SCRIPT
# -----------------------------------------------------

def train(
    model,
    dataset,
    collator,
    batch_size=16,
    lr=1e-4,
    epochs=5,
    device="cuda"
):
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collator
    )

    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=3e-4)

    pad_id = dataset.name_converter["<PAD>"]
    bos_id = dataset.name_converter["<BOS>"]

    for epoch in range(epochs):
        avg_loss = train_one_epoch(model, dataloader, optimizer, device, pad_id, bos_id)
        print(f"Epoch {epoch+1}/{epochs} | Loss = {avg_loss:.4f}")

        torch.save(model.state_dict(), f"checkpoint_epoch_{epoch+1}.pt")

    print("Training finished.")

In [4]:
# -------------------------------------------------------
# 1. Build converters
# -------------------------------------------------------
name_symbols = extract_all_symbols("../data/data.csv")
name_converter = Converter(
    symbols=name_symbols,
    special_symbols=["<PAD>", "<BOS>", "<EOS>", "<NONE>"]
)

unit_symbols = extract_all_unique_column_values("../data/data.csv", "unit")
unit_converter = Converter(
    symbols=unit_symbols,
    special_symbols=["<NONE>"]
)

tax_symbols = extract_all_unique_column_values("../data/data.csv", "tax_category")
tax_converter = Converter(
    symbols=tax_symbols,
    special_symbols=["<NONE>"]
)

# -------------------------------------------------------
# 2. Dataset + Collator
# -------------------------------------------------------
dataset = Dataset(
    "../data/data.csv",
    name_converter=name_converter,
    unit_converter=unit_converter,
    tax_converter=tax_converter
)

collator = Collator(converter=name_converter)

# -------------------------------------------------------
# 3. Build Model
# -------------------------------------------------------
model = Model(
    sym_len=len(name_converter),
    max_sam_len=max(len(x[0]) for x in dataset),      # biggest sample length
    max_nam_len=max(len(x[1]["name"]) for x in dataset),  # biggest target length
    unit_cat_len=len(unit_converter),
    tax_cat_len=len(tax_converter),
    emb_dim=256,   # or 512 if you want
    num_heads=8,
    ff_dim=1024,
    dropout=0.25,
    num_layers=4
)

# checkpoint_path = "checkpoint_epoch_1000.pt"
# checkpoint = torch.load(checkpoint_path, map_location="cuda")
# model.load_state_dict(torch.load(checkpoint_path, map_location="cuda"))


# -------------------------------------------------------
# 4. Train
# -------------------------------------------------------
train(
    model=model,
    dataset=dataset,
    collator=collator,
    batch_size=8,
    lr=1e-4,
    epochs=1000,
    device="cuda"
)

100%|██████████| 117/117 [00:05<00:00, 21.32it/s]


Epoch 1/1000 | Loss = 20.5592


100%|██████████| 117/117 [00:05<00:00, 23.18it/s]


Epoch 2/1000 | Loss = 13.8776


100%|██████████| 117/117 [00:04<00:00, 24.21it/s]


...

Epoch 618/1000 | Loss = 0.0304


100%|██████████| 117/117 [00:05<00:00, 20.77it/s]


Epoch 619/1000 | Loss = 0.0301


 37%|███▋      | 43/117 [00:02<00:03, 21.07it/s]


KeyboardInterrupt: 